In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def compute_wins(tensor):
    zeros = torch.zeros_like(tensor)
    argmax = tensor.argmax(1).reshape(-1, 1)
    zeros.scatter_(1, argmax, 1)
    wins = zeros.sum(dim=0)
    return wins

def string_to_tensor(tensor_string, softmax=False):
    # Extracts tensor values from string and returns a PyTorch tensor
    tensor_data = eval(tensor_string.split('tensor(')[1].split(', device')[0])
    t = torch.tensor(tensor_data)[:,:]
    # for S3 uncomment the line below and one above
    # t = torch.tensor(tensor_data)[:,1:]
    softmax_t = F.softmax(t, dim=1)
    wins = compute_wins(softmax_t)

    return (softmax_t if softmax else t, wins)

def extract_last_two_tensors(file_path, softmax=True):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
    except Exception as e:
        return str(e), None, None

    last_delimiter_index = content.rfind('<<<--->>>')
    if last_delimiter_index == -1:
        return f"No delimiter found in {file_path}", None, None

    last_section = content[last_delimiter_index + len('<<<--->>>'):].strip()
    lines = last_section.splitlines()
    tensors = []
    tensor_data = []
    capturing_tensor = False

    for line in lines:
        if 'tensor([' in line:
            capturing_tensor = True
            tensor_data.append(line.strip())
        elif capturing_tensor:
            tensor_data.append(line.strip())
            if 'requires_grad=True' in line:
                tensor_string = '\n'.join(tensor_data)
                tensor_obj = string_to_tensor(tensor_string, softmax=softmax)
                tensors.append(tensor_obj)
                tensor_data = []
                capturing_tensor = False

    if len(tensors) >= 2:
        return None, tensors[-2], tensors[-1]
    elif len(tensors) == 1:
        return f"Only one tensor found in {file_path}", tensors[-1], None
    else:
        return f"No tensors found after the delimiter in {file_path}", None, None

def process_directory(directory_path):
    tensor_outputs = {}
    for filename in os.listdir(directory_path):
        if filename.endswith(".log"):  # Adjust the extension based on your files
            file_path = os.path.join(directory_path, filename)
            error, tensor1, tensor2 = extract_last_two_tensors(file_path)
            tensor_outputs[filename] = {"error": error, "normal": tensor1, "reduction": tensor2}
    return tensor_outputs

In [ ]:
all_tensors = process_directory("logs/rdarts")
all_tensors

In [ ]:
{k:v for k,v in all_tensors.items() if "_s2_" in k and "lora" in k}

In [9]:
darts_dict = {k:v for k,v in all_tensors.items() if "_s4_" in k and not "lora" in k}
lora_darts_dict = {k:v for k,v in all_tensors.items() if "_s4_" in k and "lora" in k}

In [ ]:
def _get_count_list(count_dict):
    normal_count_list = []
    reduce_count_list = []
    for _, val in count_dict.items():
        normal_count_list.append(val["normal"][-1][0].item())
        reduce_count_list.append(val["reduction"][-1][0].item())
    return normal_count_list, reduce_count_list

lora_darts_normal_count, lora_darts_reduce_count = _get_count_list(lora_darts_dict)
darts_normal_count, darts_reduce_count = _get_count_list(darts_dict)

In [ ]:
print(np.mean(darts_normal_count), np.std(darts_normal_count))
print(np.mean(lora_darts_normal_count), np.std(lora_darts_normal_count))

In [ ]:
print(np.mean(darts_reduce_count), np.std(darts_reduce_count))
print(np.mean(lora_darts_reduce_count), np.std(lora_darts_reduce_count))